In [1]:
import pandas as pd
import re
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import random

from nltk.util import ngrams
from nltk import FreqDist

# Plotting tools
import seaborn as sns
import pyLDAvis
# import pyLDAvis.gensim  # don't skip this

import pyLDAvis.gensim_models as gensim
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_colwidth', -1)

In [2]:
def isNaN(num):
    return num != num

In [3]:
# Read data and preprocess
p = 0.1

df1 = pd.read_csv("humans.csv", sep = ';', index_col=None, 
                  usecols = ['screen_name', 'hashtags', 'lemma_filter','retweets_count'],
                  skiprows=lambda i: i>0 and random.random() > p
                 )
new_df = df1.reset_index().dropna()
new_df = new_df[new_df['retweets_count'] == 0]

df2 = new_df[new_df.apply(lambda x: (isNaN(x['lemma_filter'])==False) and (isNaN(x['screen_name'])==False), axis=1)]
# tweets = new_df.groupby('screen_name')['lemma_filter'].apply(' '.join).reset_index()

In [4]:
ndf0 = new_df.groupby('screen_name').size().reset_index(name='counts')
ndf1 = new_df.groupby('screen_name')['lemma_filter'].apply(' '.join).reset_index()
ndf2 = new_df.groupby('screen_name')['hashtags'].apply(','.join).reset_index()
df3_0 = pd.merge(ndf0, ndf1, on='screen_name', how='inner')
df3 = pd.merge(df3_0, ndf2, on='screen_name', how='inner')
NUM = 5 # Least number of times a user must tweet. active user
df4 = df3[df3['counts'] > NUM]
df4['hashtags_list'] = df4.apply(lambda x: [i for i in set(re.findall(r"[\w']+", x['hashtags'].lower())) if len(i) > 1], axis=1)


/Users/vikashyadav/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
id_mapping_df = df4[['screen_name', 'lemma_filter', 'hashtags_list']].reset_index()
id_mapping_df['index'] = id_mapping_df.index
id_mapping_df.to_csv('similarity_id_mapping.csv', index=False, sep=';')

In [6]:
id_mapping_df.head(2)

,index,screen_name,lemma_filter,hashtags_list
0,0,239feed,🤔🤔🤔🤔🤔 🤔🤔🤔🤔🤔 🤔🤔🤔🤔🤔 🤔🤔🤔🤔🤔 🤔🤔🤔🤔🤔 🤔🤔🤔🤔🤔,"[lovely, lovequotes, lpsdi, relationships, healthylifestyle, sarcasm, ignite, funnymemes, gorl, savagememes, memez, yummy, l4l, savage, tbt, metoo, food, lifehacks, pubity, mothlamp, puzzle, riddle, moth, propose, foodlife, memeoftheday, 239feed, canabis]"
1,1,AroundOMedia,"pretty rent pay world exist woman . become game-changing leader ? learn : quality leader ? woman tie home ? especially want work earn money ! empowerment woman one efficient tool develop society ? women´s right = fundamental human right . never forget ! always remain least little bit untamed : learn soft strong time ? let anyone break ! recognize ( domestic ) abuse violence ? learn : leadership skill teach gender : please comment dis/agree : cedaw , care ? learn : purpose istanbul convention ? learn : choose purpose perfection , ? learn : much value ? value , less allow people life value !","[feminism, metoo]"


In [7]:
def token_words(sentence):
    return [i for i in (simple_preprocess(str(sentence), deacc=True))]  # deacc=True removes punctuations

df4['processed_lemma_filter'] = df4.apply(lambda x: token_words(x['lemma_filter']), axis=1)
data = df4['processed_lemma_filter'].values

/Users/vikashyadav/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [18]:
#Build a Dictionary(associate word to numeric id)
id2word = corpora.Dictionary(data)
#print the word for id=0
print(id2word[0])

# ignore words that appear in less than 20 documents or more than 10% documents
# id2word.filter_extremes(no_below=20, no_above=0.1)
print(id2word)
print("Number of documents = {}".format(len(data)))
#Preparing Document-Term Matrix(word_id to word_freq)
corpus = [id2word.doc2bow(text) for text in data]



# Build LDA model with training data
NUM_TOPICS=10
lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=NUM_TOPICS, random_state=100, \
                                     update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

abuse
Dictionary(16814 unique tokens: ['abuse', 'agree', 'allow', 'always', 'anyone']...)
Number of documents = 468

Coherence Score:  0.5173474410846771


In [16]:
# #Build a Dictionary(associate word to numeric id)
# id2word = corpora.Dictionary(data)
# #print the word for id=0
# print(id2word[0])

# # ignore words that appear in less than 20 documents or more than 10% documents
# id2word.filter_extremes(no_below=20, no_above=0.1)
# print(id2word)
# print("Number of documents = {}".format(len(data)))
# #Preparing Document-Term Matrix(word_id to word_freq)
# corpus = [id2word.doc2bow(text) for text in data]

# # Build LDA model with training data
# NUM_TOPICS=10
# lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=NUM_TOPICS, random_state=100, \
#                                      update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

# # Compute Coherence Score
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)

In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.308480 -0.069503  1       1        42.488177
8      0.250215  0.004222  2       1        15.029801
5      0.217061  0.030111  3       1        14.799374
4     -0.036556  0.082353  4       1        6.386504 
9      0.003197  0.054267  5       1        5.926898 
7     -0.150630 -0.382351  6       1        5.427062 
6     -0.157745  0.069346  7       1        4.673123 
1     -0.151337  0.061225  8       1        2.171265 
0     -0.118387  0.087021  9       1        1.790483 
2     -0.164299  0.063309  10      1        1.307314 , topic_info=                Term         Freq        Total Category  logprob  loglift
6693   naskar         1144.000000  1144.000000  Default  30.0000  30.0000
1617   celebrate      626.000000   626.000000   Default  29.0000  29.0000
1392   brother        641.000000   641.000000   Default  28.0000  28.0000
1205   international  595.000000   595.000000   Default  27.0000  27.0000
989    walk           619.000000   619.000000   Default  26.0000  26.0000
...     ...                  ...          ...       ...      ...      ...
4470   umanrights     14.110936    14.998390    Topic10 -5.3870   4.2762 
14587  luewave        12.816808    13.704117    Topic10 -5.4832   4.2703 
4000   ike            11.813860    12.701222    Topic10 -5.5647   4.2648 
3566   etoo           96.694613    163.718982   Topic10 -3.4624   3.8106 
3157   id             28.302497    43.273327    Topic10 -4.6910   3.9126 

[504 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
6689   6      0.999355  abhijit  
1941   2      0.309619  abusive  
1941   9      0.650199  abusive  
653    4      0.997844  accessoca
14580  10     0.994321  addow    
...   ..           ...        ...
14215  10     0.976753  کوتاه    
8980   5      0.980173  आण       
6224   5      0.981230  कर       
9007   5      0.981374  सट       
4485   9      0.871810  口真帆      

[772 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 9, 6, 5, 10, 8, 7, 2, 1, 3])

In [19]:
import numpy as np
topics = lda_model.get_document_topics(corpus, per_word_topics=True)
doc_topic_matrix = np.zeros((len(data), NUM_TOPICS))

for idx, i in enumerate(topics):
    for j in i[0]:
        doc_topic_matrix[idx, j[0]] = j[1]

In [44]:
doc_topic_matrix.shape

(5227, 10)

In [20]:
cos_sim = doc_topic_matrix.dot(doc_topic_matrix.T)
cos_sim = cos_sim - np.eye(len(data))
I,J = np.where(cos_sim > 0.7)
col = np.column_stack((I,J,cos_sim[I,J]))
with open('similaritygraph.txt', 'w') as f:
    for row in col:
        if row[0] < row[1]:
            f.write('{}\t{}\t{}\n'.format(int(row[0]),int(row[1]),row[2]))

In [21]:
#!pip install infomap
!infomap similaritygraph.txt sim_output/ -N 5 --clu

  Infomap v1.0.0-beta.47 starts at [2019-04-08 07:17:17]
  -> Input network: similaritygraph.txt
  -> Output path:   sim_output/
  -> Configuration: clu
                    num-trials = 5
Parsing undirected network from file 'similaritygraph.txt'... 
  Parsing links...
  -> 78486 links
Done!
-------------------------------------
  -> 1166 nodes
  -> 78486 links with total weight 61091.1852
  -> Ordinary network input, using the Map Equation for first order network flows
Calculating global network flow using flow model 'undirected'... 
  -> Using undirected links.
  => Sum node flow: 1, sumLinkFlow: 1
Build internal network with 1166 nodes and 78486 links...
Calculating one-level codelength... done!
 -> One-level codelength: 9.24993063

Trial 1/5 starting at [2019-04-08 07:17:17]
Two-level compression: 10% 0% 
Partitioned to codelength 3.55518886e-16 + 8.29808371 + 0 = 8.298083708 in 10 modules.
Super-level compression: to codelength 8.298083708 in 10 top modules.

Recursive sub-structu

In [45]:
import pandas as pd
nodes = pd.read_csv('sim_output/similaritygraph.clu', sep=' ', header=None, skiprows=2, names=['id', 'label', 'score'])
edges = pd.read_csv('similaritygraph.txt', sep='\t', header=None, names=['src', 'dst', 'weight'])
id_mapping = pd.read_csv('similarity_id_mapping.csv', sep=';')

In [23]:
from textblob import TextBlob

ht_df = pd.read_csv('ouput.results', sep='\t', index_col='ID')[['Node', 'Label']]
ht_df['Label'] = ht_df.apply(lambda x: 1.0 if x['Label'] == 'pos' else (0.0 if x['Label'] == 'neg' else 0.5), axis=1)

ht_dic = ht_df.set_index('Node').to_dict()['Label']

#Classifier
def get_hashtag_polarity(tweet, hashtag_list):
    polarity = 0.0
    count = 0.0
    tweet_polarity = TextBlob(tweet).sentiment[0]
    for ht in hashtag_list:
        if ht in ht_dic:
            polarity += ht_dic[ht]
            count += 1.0
    if count > 0:
        polarity = polarity/count
        result_polarity = (tweet_polarity+polarity)/2.0
    else:
        result_polarity = tweet_polarity
    return [polarity, result_polarity, 'pos' if result_polarity > 0.6 else ('neg' if result_polarity < 0.4 else 'neutral')]
     

In [24]:
id_mapping['sentiment'] = .apply(lambda x: get_hashtag_polarity(x['lemma_filter'], x['hashtags_list']), axis=1)

In [25]:
nodesdf = pd.merge(nodes, id_mapping[['index', 'sentiment']], left_on=['id'], right_on=['index'], how='inner')

In [26]:
nodesdf['sentiment'] = nodesdf.apply(lambda x: x['sentiment'][2], axis=1)

In [31]:
nodesdf.drop('index', axis=1).to_csv('similarity_graph_nodes.csv', sep=',', header=True, index=None)

In [32]:
edges.to_csv('similarity_graph_edges.csv', sep=',', header=True, index=None)

In [34]:
nodesdf.head(5)

,id,label,score,index,sentiment
0,0,0,0.000018,0,neg
1,1,0,0.000023,1,neg
2,12,0,0.001079,12,neg
3,24,0,0.001027,24,neutral
4,26,0,0.000672,26,neg


In [35]:
edges.head(5)

,src,dst,weight
0,0,587,0.717159
1,0,830,0.717237
2,0,3118,0.705501
3,1,587,0.719873
4,1,830,0.719911


In [46]:
id_mapping

index              screen_name  \
0     0      1000_acts                 
1     1      100womeniknow             
2     2      12inchchurro              
3     3      16tage.ch                 
4     4      1718bush                  
5     5      17contra_o_sistema        
6     6      1_.o1o._1                 
7     7      1_more_voice              
8     8      1_per3                    
9     9      1happyinfidel             
10    10     1in6org                   
11    11     1jefftuck                 
12    12     1mindfulmommy             
13    13     1sthcc                    
14    14     234radio                  
15    15     239feed                   
16    16     27thhgraphics             
17    17     2blondesmedia             
18    18     2to1_show                 
19    19     2wokegirlss               
20    20     39smiles                  
21    21     3dOLLabILL                
22    22     3rdfingerpointedatme      
23    23     4chan.arsenal.k           
24    24     4selfmotivationfitness    
25    25     504thepeople              
26    26     50truth                   
27    27     5imba_ali                 
28    28     5star_princess_1          
29    29     5thwavecollective         
...   ..                   ...         
5197  5197   yourfacemyhole            
5198  5198   yourmomppbiggerthanmine   
5199  5199   yourpleasureprincipal     
5200  5200   yoursistersspace          
5201  5201   ypphrdivas                
5202  5202   yurasis.political         
5203  5203   yusef.hoe                 
5204  5204   yvonnevangalen            
5205  5205   ywcalethbridge            
5206  5206   ywcalgary                 
5207  5207   yxelle                    
5208  5208   zahodismelo               
5209  5209   zainab4raleigh            
5210  5210   zajkouskoo                
5211  5211   zakharova_galaz           
5212  5212   zaphtik                   
5213  5213   zarajsargent              
5214  5214   zareta_dzeboeva           
5215  5215   zayka_shop.ua             
5216  5216   zeerajasthannews          
5217  5217   zeke_thomas               
5218  5218   zerosviowa                
5219  5219   zestfulfeminists          
5220  5220   zetetic_news_network      
5221  5221   zizimmature               
5222  5222   zoyla_contreras           
5223  5223   zubaanbooks               
5224  5224   zuckerwaffe               
5225  5225   zyanyabarcelo             
5226  5226   zzzjjj89060ix             

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     